# 発話感情分類モデルの作成

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch

/opt/conda/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# データの準備

## データダウンロード

In [2]:
!rm -r ./data

In [3]:
!git clone https://github.com/ids-cv/wrime.git ./data/

Cloning into './data'...
remote: Enumerating objects: 58, done.
remote: Counting objects: 100% (58/58), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 58 (delta 20), reused 48 (delta 13), pack-reused 0
Unpacking objects: 100% (58/58), done.


In [4]:
!rm -r ./data/.git

In [5]:
!wget -O ./data/japanese_empathetic_dialogues.xlsx -P content  https://www.dropbox.com/s/rkzyeu58p48ndz3/japanese_empathetic_dialogues.xlsx?dl=0

--2022-07-30 13:46:41--  https://www.dropbox.com/s/rkzyeu58p48ndz3/japanese_empathetic_dialogues.xlsx?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.80.18, 2620:100:6030:18::a27d:5012
Connecting to www.dropbox.com (www.dropbox.com)|162.125.80.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/rkzyeu58p48ndz3/japanese_empathetic_dialogues.xlsx [following]
--2022-07-30 13:46:41--  https://www.dropbox.com/s/raw/rkzyeu58p48ndz3/japanese_empathetic_dialogues.xlsx
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc78834bc3210eccf23b02adc924.dl.dropboxusercontent.com/cd/0/inline/BqADomMflA4HHssZEsC-sGMG6jfxVyO9UNr1SCBWJVaCuYuR-9kjj_PSliNBa4r-0tTJ-KaJ3rDFr-3ffzoieiJj4b63fhYQ16JVs8bELUoqKnotBcA0Cka-_5_XKtp8NYPzYQzdClaIKd8OkwyIfWHMphmGvzztKZiTQ0q_MzPDFA/file# [following]
--2022-07-30 13:46:42--  https://uc78834bc3210eccf23b02adc924.dl.dropboxusercontent.com/cd/0/i

## データ前処理

## japanese_empathetic_dialogues前処理

In [6]:
df_env = pd.read_excel("./data/japanese_empathetic_dialogues.xlsx",sheet_name="状況文")
df_env = df_env.rename(columns={'作業No':'ID'})

In [7]:
df_utt = pd.read_excel("./data/japanese_empathetic_dialogues.xlsx",sheet_name="対話")

In [8]:
df = pd.merge(df_utt, df_env, how="left", on="ID")

In [9]:
empathtic_names = df["感情"].unique()
print(empathtic_names)
print(len(empathtic_names))
print(df["感情"].value_counts())

['おどろく' 'わくわくする' '怒る' '誇りに思う' '悲しい' 'いらいらする' '感謝する' 'さびしい' '怖い' '恐ろしい'
 'うしろめたい' '感動する' '嫌悪感を抱く' '期待する' '自信がある' '激怒する' '不安に思う' '待ち望む' '楽しい'
 '懐かしい' 'がっかりする' '心構えする' '羨ましい' '満足' '打ちのめされる' '恥ずかしい' '思いやりを持つ' '感傷的になる'
 '信頼する' '恥じる' '懸念する' '誠実な気持ち']
32
おどろく       2500
わくわくする     2500
懸念する       2500
恥じる        2500
信頼する       2500
感傷的になる     2500
思いやりを持つ    2500
恥ずかしい      2500
打ちのめされる    2500
満足         2500
羨ましい       2500
心構えする      2500
がっかりする     2500
懐かしい       2500
楽しい        2500
待ち望む       2500
不安に思う      2500
激怒する       2500
自信がある      2500
期待する       2500
嫌悪感を抱く     2500
感動する       2500
うしろめたい     2500
恐ろしい       2500
怖い         2500
さびしい       2500
感謝する       2500
いらいらする     2500
悲しい        2500
誇りに思う      2500
怒る         2500
誠実な気持ち     2500
Name: 感情, dtype: int64


In [10]:
# 話者がAだけ残す
df = df[df["話者"] == "A"]

In [11]:
df.drop(["ID", "話者", "状況文"], axis=1,inplace=True)
df = df.reindex(columns=['発話','感情'])
df[50:85]

,発話,感情
100,今日ズボンのチャックが開いたまま大勢の前でスピーチをして、恥ずかしい思いをしたよ。,恥ずかしい
102,スピーチのことに集中していたから気づかなかったんだよ。,恥ずかしい
104,今朝混んでる電車に息切れをしたおじいさんが乗ってきたから席を譲ったんだ。,思いやりを持つ
106,勇気を出して声をかけたら感謝をされたよ。,思いやりを持つ
108,若いうちに親を亡くした子供の映画を見て感傷的になった。,感傷的になる
110,今回の映画は主人公の描写が繊細でグッとくるものがあったよ。,感傷的になる
112,今度の大きなプロジェクト、賢くて才能のある若手に任せることにしました。,信頼する
114,色々な事を知っているし、落ち着いて冷静な判断もできる信頼しがいのある男です。,信頼する
116,この間雑談中に知ったかぶりしているのがバレちゃった。,恥じる
118,顔に全部出ていてみんなお見通しだったって言っていた。,恥じる


In [12]:
# 32個の感情を8個にまとめる
emotions_dict = {
    "喜び":["感謝する","感動する","楽しい","満足"],
    "悲しみ":["悲しい","さびしい","がっかりする","打ちのめされる","感傷的になる"],
    "期待":["わくわくする","期待する","待ち望む"],
    "驚き":["おどろく"],
    "怒り":["怒る","いらいらする","激怒する"],
    "恐れ":["怖い","恐ろしい","不安に思う","懸念する"],
    "嫌悪":["うしろめたい","嫌悪感を抱く","恥ずかしい","恥じる"],
    "信頼":["自信がある","信頼する","誠実な気持ち"]
}
drop_emotions = ["誇りに思う","心構えする","羨ましい","懐かしい","思いやりを持つ"]

In [13]:
for p_emo, c_emos in emotions_dict.items():
    df["感情"].replace(c_emos,p_emo,inplace=True)

In [14]:
# 使用しない感情の行を削除する
for emo in drop_emotions:
    df = df[df["感情"] != emo]

In [15]:
empathtic_names = df["感情"].unique()
print(empathtic_names)
print(len(empathtic_names))
print(df["感情"].value_counts())

['驚き' '期待' '怒り' '悲しみ' '喜び' '恐れ' '嫌悪' '信頼']
8
悲しみ    6250
喜び     5000
恐れ     5000
嫌悪     5000
期待     3750
怒り     3750
信頼     3750
驚き     1250
Name: 感情, dtype: int64


## WRIME前処理

In [16]:
df_wrime = pd.read_csv('./data/wrime-ver2.tsv', delimiter='\t')
# 必要な列だけ抽出
df_wrime = df_wrime.loc[:,["Sentence","Train/Dev/Test",
"Writer_Joy","Writer_Sadness","Writer_Anticipation","Writer_Surprise","Writer_Anger","Writer_Fear","Writer_Disgust","Writer_Trust","Writer_Sentiment",
"Avg. Readers_Joy","Avg. Readers_Sadness","Avg. Readers_Anticipation","Avg. Readers_Surprise","Avg. Readers_Anger","Avg. Readers_Fear",
"Avg. Readers_Disgust","Avg. Readers_Trust","Avg. Readers_Sentiment"]]
len(df_wrime.columns)

20

In [17]:
# 2人のアノテーターの合計の多数決で感情を決定
# df_wrime.isnull().sum()
add_emotion_dict = {
    "喜び":["Writer_Joy", "Avg. Readers_Joy"],
    "悲しみ":["Writer_Sadness","Avg. Readers_Sadness"],
    "期待":["Writer_Anticipation","Avg. Readers_Anticipation"],
    "驚き":["Writer_Surprise","Avg. Readers_Surprise"],
    "怒り":["Writer_Anger","Avg. Readers_Anger"],
    "恐れ":["Writer_Fear","Avg. Readers_Fear"],
    "嫌悪":["Writer_Disgust","Avg. Readers_Disgust"],
    "信頼":["Writer_Trust","Avg. Readers_Trust"]
    # "Sentiment":["Writer_Sentiment","Avg. Readers_Sentiment"] （今回は感情極性を使わない）
}

# それぞれの感情で合計値を計算
for emo_p, emo_c_list in add_emotion_dict.items():
    df_wrime = pd.concat([df_wrime, pd.DataFrame(df_wrime.loc[:,emo_c_list].sum(axis=1), columns=[emo_p])],axis=1)

In [18]:
df_wrime = df_wrime.loc[:, ["Sentence"] + list(add_emotion_dict.keys())] # 必要な列だけ抽出
df_wrime.rename(columns={"Sentence":"発話"}, inplace=True) # 列名変更

In [19]:
# 感情ラベルを数値で多数決してラベルを決定
df_wrime = pd.concat([df_wrime, pd.DataFrame(df_wrime.loc[:, list(add_emotion_dict.keys())].idxmax(axis=1), columns=["感情"])], axis=1) # 各感情で一番大きい感情を取り出し新しく感情ラベル列を作成

In [20]:
df_wrime = df_wrime.loc[:, ["発話","感情"]] # 使用する列だけ抽出
df_wrime

,発話,感情
0,ぼけっとしてたらこんな時間。チャリあるから食べにでたいのに…,悲しみ
1,今日の月も白くて明るい。昨日より雲が少なくてキレイな〜 と立ち止まる帰り道。チャリなし生活も...,喜び
2,早寝するつもりが飲み物がなくなりコンビニへ。ん、今日、風が涼しいな。,驚き
3,眠い、眠れない。,悲しみ
4,ただいま〜 って新体操してるやん!外食する気満々で家に何もないのに!テレビから離れられない…!,喜び
...,...,...
34995,真夜中にふと思い立ち、ノートPCを持って部屋を出て、ダイニングで仕事したらすんごい捗った。\...,喜び
34996,ぐっどこんでぃしょん。\n心も頭もクリア。\n秋分の日のおかげかな？\n人と自然としっとり過...,喜び
34997,朝から免許の更新へ。\n90分で終わり、出口へ向かうと献血の呼びかけが。\nみんな通り過ぎて...,喜び
34998,夜も更けて参りましたが、食後のコーヒーが飲みたいのでドリップ開始…\n\nぼんやり秋の夜長を...,期待


In [21]:
print(len(df_wrime["感情"].unique()))
print(df_wrime["感情"].value_counts())

8
喜び     10095
悲しみ     8528
期待      6926
驚き      4046
恐れ      2225
嫌悪      1911
怒り       830
信頼       439
Name: 感情, dtype: int64


## データフレームの結合

In [22]:
print(len(df))
print(len(df_wrime))
df = pd.concat([df, df_wrime], axis=0)
print(len(df))

33750
35000
68750


In [23]:
 # シャッフルする
df = df.sample(frac=1, random_state=0, ignore_index=True)

In [24]:
print(len(df["感情"].unique()))
print(df["感情"].value_counts())

8
喜び     15095
悲しみ    14778
期待     10676
恐れ      7225
嫌悪      6911
驚き      5296
怒り      4580
信頼      4189
Name: 感情, dtype: int64


In [25]:
df.sample(frac=1)

,発話,感情
17181,いや、さすがにみんな立ち回り下手すぎた笑,驚き
8297,昨日と同じ場所にねこちゃんがいてくれるのうれしいよなぁ,喜び
37828,こんな調子で将来どうやって生きていくの？⇒心配しなくてもその将来が来る前に黒い染みになります,悲しみ
13201,ふっじっさーん ふっじっさーん゛゛,喜び
63145,何も悪いことをしていないのに、とんでもない話です。,怒り
...,...,...
32078,わたしの家の前は広場になっているんですが、そこでいつも立ち話をするおばさんがいます。,怒り
49852,バス間に合ったーパフェでそう,喜び
55121,3日くらい前にスピループスでペンを買ったんだが、ログインせずに情報だけ打ち込んだせいなのかメ...,恐れ
18376,さあもうすぐ沢山のリア充が乗り込んで来るぞ！皆覚悟はよいか!?,悲しみ


## 発話テキスト前処理

In [26]:
import re

In [27]:
def text_preprocessing(text):
    # 「]の削除
    text = text.replace('「','')
    text = text.replace('」','')
    # URLの削除
    text = re.sub(r'http?://[\w/:%#\$&\?\(\)~\.=\+\-]+', '', text)
    text = re.sub(r'https?://[\w/:%#\$&\?\(\)~\.=\+\-]+', '', text)
    # pic.twitter.comXXXの削除
    text = re.sub(r'pic.twitter.com/[\w/:%#\$&\?\(\)~\.=\+\-]+', '', text)
    # 全角記号削除
    text = re.sub("[\uFF01-\uFF0F\uFF1A-\uFF20\uFF3B-\uFF40\uFF5B-\uFF65\u3000-\u303F]", '', text)
    # 半角記号の置換
    text = re.sub(r'[!-/:-@[-`{-~]', r' ', text)
    # 全角記号の置換 (ここでは0x25A0 - 0x266Fのブロックのみを除去)
    text = re.sub(u'[■-♯]', ' ', text)
    # 数値をすべて0に変換
    text = re.sub(r'\d+', '0', text)
    text = text.replace("\n","")
    text = text.replace("。","")
    text = text.replace(".","")
    text = text.replace(",","")
    text = text.lower()
    return text

In [28]:
df["発話"] = df["発話"].map(text_preprocessing)

In [29]:
df.sample(frac=1)

,発話,感情
3481,最近のホロライブは内部コラボを重視してる前ほど外部コラボはなくなった,悲しみ
51852,昼暑くて朝寒いし朝明るくて夜暗いし夜寝たくなくて昼眠いし繰り返してばっかかよ,悲しみ
47291,いろんな経験をしてきたんやな,驚き
3171,ラジオ聴く環境作らなくなったし校長教頭変わってからほっとんど聴いてない奴いうのもアレなんだけ...,信頼
34026,そうなんですよ気になり出すとずっと気になっちゃって,怒り
...,...,...
23823,今0歳保育園でみんなで描いたみたいよ,喜び
43365,茶色いとか煮物が嫌だとか母に文句を言っていた昔の自分が恥ずかしい,嫌悪
43620,バーゲンだから買い物してくるよ,期待
41359,本当に怖いよねこれからどうなってくか不安,恐れ


## データの分割

In [31]:
from sklearn.model_selection import train_test_split

In [32]:
df = df.rename(columns={'感情':'label'})
df = df.rename(columns={'発話':'text'})

In [44]:
data_train, data_test = train_test_split(df, random_state=111, stratify=df.label) # 訓練用とテスト用に分割 defalut 25%がテストデータ
print(len(data_train))
print(data_train["label"].value_counts() /  len(data_train))
print(len(data_test))
print(data_test["label"].value_counts() / len(data_test))

51562
4    0.219561
3    0.214945
1    0.155289
5    0.105097
6    0.100520
0    0.077033
2    0.066619
7    0.060936
Name: label, dtype: float64
17188
4    0.219572
3    0.214976
1    0.155283
5    0.105073
6    0.100535
0    0.077030
2    0.066616
7    0.060915
Name: label, dtype: float64


In [45]:
train_docs = data_train["text"].tolist()
train_labels = data_train["label"].tolist()
len(train_docs)

51562

In [46]:
test_docs = data_test["text"].tolist()
test_labels = data_test["label"].tolist()
len(test_docs)

17188

# モデル構築

In [47]:
# GPU が利用できる場合は GPU を利用する

device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

'cuda:0'

In [49]:
from transformers import BertForSequenceClassification,BertJapaneseTokenizer
from transformers import AdamW

sc_model = BertForSequenceClassification.from_pretrained("cl-tohoku/bert-base-japanese-v2", num_labels=len(empathtic_names))
model = sc_model.to(device)
tokenizer = BertJapaneseTokenizer.from_pretrained("cl-tohoku/bert-base-japanese-v2")

Downloading: 100%|██████████| 517/517 [00:00<00:00, 197kB/s]
Downloading: 100%|██████████| 427M/427M [00:05<00:00, 78.6MB/s] 
Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-v2 were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a Bert

# エンコーディング
Transformerモデルの入力としてはテンソル形式に変換する必要がある。
返り値のテンソルのタイプを選ぶことができる。ここではPyTorchのテンソル型で返してくれるよう、return_tensors='pt'としている。


In [50]:
train_encodings = tokenizer(train_docs, return_tensors='pt', padding=True, truncation=True, max_length=128).to(device)
test_encodings = tokenizer(test_docs, return_tensors='pt', padding=True, truncation=True, max_length=128).to(device)

In [51]:
import torch

class JpSentiDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = JpSentiDataset(train_encodings, train_labels)
test_dataset = JpSentiDataset(test_encodings, test_labels)

## 評価関数の設定

In [52]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted', zero_division=0)
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

## トレーニング

In [53]:
!mkdir ./logs
!ls

JEmpatheticDialogues_WRIME.ipynb  data	docker_env  logs


In [54]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=4,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    save_total_limit=1,              # limit the total amount of checkpoints. Deletes the older checkpoints.
    dataloader_pin_memory=False,  # Whether you want to pin memory in data loaders or not. Will default to True
    # evaluation_strategy="epoch",     # Evaluation is done at the end of each epoch.
    evaluation_strategy="steps",
    logging_steps=50,
    logging_dir='./logs'
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    tokenizer=tokenizer,
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset,             # evaluation dataset
    compute_metrics=compute_metrics  # The function that will be used to compute metrics at evaluation
)

trainer.train()

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 51562
  Num Epochs = 4
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 12892
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
50,2.084600,2.010182,0.186060,0.140976,0.151905,0.186060
100,1.982100,1.913849,0.260182,0.157552,0.194930,0.260182
150,1.872600,1.776684,0.358273,0.275931,0.267133,0.358273
200,1.755200,1.643520,0.409239,0.341711,0.483181,0.409239
250,1.580500,1.561088,0.438213,0.401521,0.447354,0.438213
300,1.562100,1.459406,0.489993,0.464146,0.521146,0.489993
350,1.531000,1.413880,0.505178,0.491580,0.535284,0.505178
400,1.463700,1.428174,0.508843,0.488556,0.540471,0.508843
450,1.418800,1.418743,0.504189,0.492892,0.559356,0.504189
500,1.365600,1.392136,0.515418,0.510849,0.533314,0.515418


***** Running Evaluation *****
  Num examples = 17188
  Batch size = 64
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':
***** Running Evaluation *****
  Num examples = 17188
  Batch size = 64
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':
***** Running Evaluation *****
  Num examples = 17188
  Batch size = 64
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires

TrainOutput(global_step=12892, training_loss=0.796463491603746, metrics={'train_runtime': 8566.2152, 'train_samples_per_second': 24.077, 'train_steps_per_second': 1.505, 'total_flos': 1.1765360958244992e+16, 'train_loss': 0.796463491603746, 'epoch': 4.0})

## 評価

In [55]:
trainer.evaluate(eval_dataset=test_dataset)

***** Running Evaluation *****
  Num examples = 17188
  Batch size = 64
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


{'eval_loss': 1.6394577026367188,
 'eval_accuracy': 0.6062950895973935,
 'eval_f1': 0.6059304137567896,
 'eval_precision': 0.6066650551917165,
 'eval_recall': 0.6062950895973935,
 'eval_runtime': 28.2961,
 'eval_samples_per_second': 607.433,
 'eval_steps_per_second': 9.507,
 'epoch': 4.0}

## モデルの保存

In [56]:
save_directory = "./JEmpatheticDialogues_WRIME_not_same_labels_num"

tokenizer.save_pretrained(save_directory)
model.save_pretrained(save_directory)

tokenizer config file saved in ./JEmpatheticDialogues_WRIME_not_same_labels_num/tokenizer_config.json
Special tokens file saved in ./JEmpatheticDialogues_WRIME_not_same_labels_num/special_tokens_map.json
Configuration saved in ./JEmpatheticDialogues_WRIME_not_same_labels_num/config.json
Model weights saved in ./JEmpatheticDialogues_WRIME_not_same_labels_num/pytorch_model.bin


## 学習グラフ

In [63]:
%load_ext tensorboard
%tensorboard --logdir logs --host localhost --port 8888

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
